# Using RAGAS to evaluate your RAG system
- **RAGAS** is a strong Python module used to evaluate the RAG systems. One of the most popular techniques in RAG evaluation is **model-based evaluation**. This method uses a powerful/commercial LLM model to measure the similarity between the RAG’s generated responses and the expected results.
- I use **Google VertexAI** and their `gemini-1.5-flash` LLM model to evaluate my RAG system in this lab.
- The main components in this system include:
  - LLM model: `DeepSeek R1 Distill LLama3 70B`
  - Embedding model: `sentence-transformers/paraphrase-multilingual-mpnet-base-v2`
  - Evaluation LLM model: `gemini-1.5-flash`

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os

In [4]:
sys.path

['/Users/cuongdm8499/Me/git-cuongpiger/llm-evaluation',
 '/Users/cuongdm8499/Applications/PyCharm Professional Edition.app/Contents/plugins/python-ce/helpers/pydev',
 '/Users/cuongdm8499/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers-pro/jupyter_debug',
 '/opt/anaconda3/envs/llm-evaluation/lib/python310.zip',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10/lib-dynload',
 '',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/setuptools/_vendor']

## Prepare the documents

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from collections import defaultdict
from langchain.schema import Document

In [6]:
directory_path = "./../data/vks/pdf/vi"  # Change this to your directory
md_files = []

for root, _, files in os.walk(directory_path):
    for file in files:
        if file.endswith(".pdf"):
            md_files.append(os.path.join(root, file))

print(md_files)

['./../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf', './../data/vks/pdf/vi/02-nlb.pdf', './../data/vks/pdf/vi/04-thong-bao-cap-nhat-release-note.pdf', './../data/vks/pdf/vi/01-vks-la-gi.pdf']


In [7]:
docs = []

for file in md_files:
    loader = PyPDFLoader(file)

    data = loader.load()
    docs.extend(data)

docs

[Document(metadata={'producer': 'Skia/PDF m133', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 'creationdate': '2025-02-18T07:22:18+00:00', 'title': '02-mo-hinh-hoat-dong', 'moddate': '2025-02-18T07:22:18+00:00', 'source': './../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='3. Mô hình hoạt động của VKS\nVNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public Cluster và Private\nCluster. Đối với node group, VKS hỗ trợ 2 loại node group chính là Public Node Group và\nPrivate Node Group. Từ đó, khách hàng có thể kết hợp giữa các loại cluster và node group để\ntạo ra các mô hình hoạt động phù hợp với yêu cầu của ứng dụng, cụ thể:\nPublic Cluster với Public Node Group: Các node trong nodegroup có địa chỉ IP Public\nriêng, kết đến nối ControlPlane thông qua IP Public của Kubernetes API Server. Điều này\ndẫn rủi ro bảo mật cao hơn. T

In [8]:
# Group documents by source
merged_docs = defaultdict(str)
for doc in docs:
    merged_docs[doc.metadata["source"]] += (
            doc.page_content + "\n"
    )  # Concatenating with newline

In [9]:
new_docs = []

for source, doc in merged_docs.items():
    new_doc = Document(page_content=doc, metadata={"source": source})

    new_docs.append(new_doc)

In [10]:
new_docs

[Document(metadata={'source': './../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf'}, page_content='3. Mô hình hoạt động của VKS\nVNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public Cluster và Private\nCluster. Đối với node group, VKS hỗ trợ 2 loại node group chính là Public Node Group và\nPrivate Node Group. Từ đó, khách hàng có thể kết hợp giữa các loại cluster và node group để\ntạo ra các mô hình hoạt động phù hợp với yêu cầu của ứng dụng, cụ thể:\nPublic Cluster với Public Node Group: Các node trong nodegroup có địa chỉ IP Public\nriêng, kết đến nối ControlPlane thông qua IP Public của Kubernetes API Server. Điều này\ndẫn rủi ro bảo mật cao hơn. Tuy nhiên bạn hoàn toàn có thể hạn chế truy cập đến\nControlPlane thông qua tính năng Whitelist IP. Một điều lưu ý là bạn cần đảm bảo không\nhạn chế truy cập từ các IP của các node trong nodegroup.\nPublic Cluster với Private Node Group: Các node trong nodegroup không có địa chỉ IP\nPublic riêng, không thể kết nối trực tiếp đế

In [11]:
len(new_docs)

4

## Chunking and save into vector-store database

In [12]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

In [13]:
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)
vector_store = InMemoryVectorStore(embeddings_model)

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
chunk_size = 7000
chunk_overlap = 6800
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

In [15]:
splits = text_splitter.split_documents(new_docs)

In [16]:
vector_store.add_documents(splits)  # save documents into vectorstore

['7f580c0d-657a-4b75-b4f2-2a257f20f5c4',
 '49cc6920-a5d7-4767-ab4c-d2eacd3acbfd',
 'e9ac6cdd-babe-4745-ab9d-31ba11bfb1ca',
 '78f7f007-e21e-4b2a-9b1f-c91c3aae1831',
 '602af056-88a4-45ef-9b6d-f8e125f9a090',
 '043e44de-e014-4d1a-b016-774d14129dfb',
 'c6970d07-77b3-4906-abc8-e7008ae901d9',
 '9e0942b0-566a-4763-a130-c94f5b931331',
 '1ebf6623-c68a-458b-a9dd-7618cad176c3',
 '0f4d84b9-2c54-4371-885f-1a1104220f88',
 '52f3720d-d852-4d0d-badd-b7c85a3e54c3',
 'c073a93a-6ba0-46d5-af80-e2449f9c0b57',
 '32daed41-1304-4af0-a0eb-c5818c8390d6',
 '40df092c-7ff5-426d-8ac7-429ed37aec57',
 '884cb838-ab0d-4c1d-8756-d7ab5758407a',
 '394af565-3803-4e26-8065-899844888570',
 'd621200e-ffff-46c4-a886-c590541eb10c',
 '963d97a8-f20b-486a-8572-60269c6738ab',
 'd7828e37-b7c5-40a6-95fa-b88f9c4e8450',
 '8d7aa25c-d5cc-4c7a-ad3c-cec75858d5a5',
 '1df4eea3-6699-4b72-a22a-752df8698928',
 'ac39dd1c-0378-4894-8ced-9710beaddcce',
 'be15eac9-3761-4e51-977f-5d37c14a37a5',
 '1ae8a225-8bf4-475c-b396-ce25fa567d5d',
 'a3e72ef9-b4a1-

In [17]:
retriever = vector_store.as_retriever()

In [18]:
query = "VKS là gì?"
for i, doc in enumerate(retriever.invoke(query)):
    print(f"Document {i}:\n{doc}", end="\n\n")

Document 0:
page_content='1. Giới thiệu về VNGCloud Kubernetes
Service - VKS
1.1. VNGCloud Kubernetes Service (VKS) là gì?
VNGCloud Kubernetes Service (hay còn gọi là VKS) là một dịch vụ quản lý Kubernetes được cung
cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách
dễ dàng và hiệu quả. VKS giúp bạn tập trung vào việc phát triển ứng dụng mà không cần quan tâm
đến việc quản lý Control Plane của Kubernetes.
VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn
giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền
tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các
ứng dụng container trên môi trường phân tán.
1.2. Những điểm nổi bật của VKS
Những điểm nổi bật của dịch vụ VKS có thể kể đến gồm:
Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng
bạn khỏi gánh nặng quản lý Control Plane của Ku

## Configure some environment variables

In [24]:

from modules import utils

In [25]:
ENV_FILE_PATH = "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
MAX_TOKENS = 32700

envs = utils.load_env_to_dict(ENV_FILE_PATH)

## Setup RAG chain

In [26]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [27]:
llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=envs['VLLM_HOST_URL_2'] + "/v1/",
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    streaming=True,
    model_kwargs={"stop": ["<|eot_id|>",'<|eom_id|>']},
)

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'stop'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [29]:
# Define prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
# Setup RAG pipeline
rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)


## Preparing the Evaluation Data

In [30]:
from ragas import EvaluationDataset, SingleTurnSample

In [31]:
questions = ["VKS có những tính năng gì nổi bật?",
             "VKS là gì?",
             "So sánh public và private cluster trong VKS",
             ]

ground_truths = [
    """
    Dịch vụ VKS (VNGCloud Kubernetes Service) có những tính năng nổi bật sau:

    Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS giúp bạn giải phóng khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.
    Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.
    Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.
    Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng.
    Scaling & Healing Automatically: VKS tự động mở rộng Node group khi cần thiết và tự động sửa lỗi khi node gặp vấn đề, giúp bạn tiết kiệm thời gian và công sức quản lý.
    Giảm chi phí và nâng cao độ tin cậy: VKS triển khai Control Plane của Kubernetes ở chế độ sẵn sàng cao và hoàn toàn miễn phí.
    Tích hợp Blockstore Native (Container Storage Interface - CSI): VKS cho phép bạn quản lý Blockstore thông qua YAML của Kubernetes, cung cấp lưu trữ bền vững cho container.
    Tích hợp Load Balancer (Network Load Balancer, Application Load Balancer): VKS cung cấp khả năng quản lý NLB/ALB thông qua YAML của Kubernetes, giúp bạn dễ dàng expose Service trong Kubernetes ra bên ngoài.
    Nâng cao bảo mật: VKS cho phép bạn tạo Private Node Group với chỉ Private IP và kiểm soát quyền truy cập vào cluster.
    Ngoài ra, VKS còn có các ưu điểm sau:

    Dễ sử dụng: VKS cung cấp giao diện đơn giản và dễ sử dụng.
    Chi phí hợp lý: VKS cung cấp mức giá cạnh tranh cho các dịch vụ của mình.
    Bạn có muốn tìm hiểu thêm về tính năng nào cụ thể của VKS không?
    """,
    """
    VKS là viết tắt của VNGCloud Kubernetes Service.

    Nó là một dịch vụ quản lý Kubernetes được cung cấp bởi VNGCloud.

    VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách dễ dàng và hiệu quả. Nó giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

    Nói cách khác, VKS là một giải pháp giúp bạn dễ dàng sử dụng Kubernetes mà không cần phải tự cài đặt và quản lý nó.
    """,
    """
    Public Cluster và Private Cluster là hai loại cụm Kubernetes chính được hỗ trợ bởi VKS. Mỗi loại có ưu nhược điểm riêng, phù hợp với những nhu cầu bảo mật và sử dụng khác nhau.

    Dưới đây là bảng so sánh chi tiết giữa Public Cluster và Private Cluster trong VKS:

    Tính năng	Public Cluster	Private Cluster
    Kết nối	Sử dụng địa chỉ Public IP để giao tiếp giữa nodes, control plane, client và các dịch vụ khác trong VNG Cloud	Sử dụng địa chỉ Private IP để giao tiếp giữa nodes, control plane, client và các dịch vụ khác trong VNG Cloud
    Bảo mật	Bảo mật trung bình do các kết nối sử dụng Public IP	Bảo mật cao hơn với tất cả kết nối đều private và giới hạn truy cập
    Quản lý truy cập	Khó kiểm soát hơn, có thể quản lý truy cập thông qua tính năng Whitelist	Kiểm soát truy cập chặt chẽ, mọi kết nối đều nằm trong mạng private của VNG Cloud
    Khả năng mở rộng (AutoScaling)	Dễ dàng mở rộng thông qua tính năng Auto Scaling	Dễ dàng mở rộng thông qua tính năng Auto Scaling
    Khả năng tự hồi phục (AutoHealing)	Tự động phát hiện lỗi và khởi động lại node (Auto Healing)	Tự động phát hiện lỗi và khởi động lại node (Auto Healing)
    Khả năng truy cập từ bên ngoài	Dễ dàng truy cập từ bất kỳ đâu	Truy cập từ bên ngoài phải qua các giải pháp bảo mật khác
    Cấu hình và triển khai	Đơn giản hơn do không yêu cầu thiết lập mạng nội bộ	Phức tạp hơn, yêu cầu cấu hình mạng private và bảo mật
    Chi phí	Thường thấp hơn do không cần thiết lập cơ sở hạ tầng bảo mật phức tạp	Chi phí cao hơn do yêu cầu thêm các thành phần bảo mật và quản lý
    Tính linh hoạt	Cao, dễ dàng thay đổi và truy cập các dịch vụ	Linh hoạt hơn trong các ứng dụng yêu cầu bảo mật, nhưng ít linh hoạt hơn cho các ứng dụng yêu cầu truy cập từ bên ngoài
    Phù hợp với	Các ứng dụng không yêu cầu bảo mật cao và cần sự linh hoạt, truy cập từ nhiều địa điểm	Các ứng dụng yêu cầu bảo mật cao, tuân thủ nghiêm ngặt các quy định về bảo mật và quyền riêng tư
    Kết luận:

    Public Cluster phù hợp cho các ứng dụng không yêu cầu bảo mật cao và cần sự linh hoạt, truy cập từ nhiều địa điểm. Dễ dàng triển khai và quản lý nhưng có rủi ro bảo mật cao hơn.
    Private Cluster phù hợp cho các ứng dụng yêu cầu bảo mật cao, tuân thủ nghiêm ngặt các quy định về bảo mật và quyền riêng tư. Mang lại kết nối ổn định và bảo mật, nhưng yêu cầu cấu hình và quản lý phức tạp hơn, cũng như chi phí cao hơn.
    Bạn cần cân nhắc kỹ lưỡng nhu cầu của mình để lựa chọn loại cluster phù hợp nhất.
    """
]

answers = []
contexts = []

for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.invoke(query)])

# Convert dict to dataset
dataset = EvaluationDataset(samples=[SingleTurnSample(
    user_input=questions[i],
    retrieved_contexts=contexts[i],
    response=answers[i],
    reference=ground_truths[i]
) for i in range(len(questions))])

# Prepare the evaluator model using VertexAI

In [33]:
from langchain_google_vertexai import ChatVertexAI

In [34]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    # "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
    "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [35]:
evaluator_llm = ChatVertexAI(model_name="gemini-1.5-flash", temperature=0)

## Evaluating the RAG application


In [36]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


In [37]:
result = evaluate(
    llm=evaluator_llm,
    embeddings=embeddings_model,
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

Evaluating: 100%|██████████| 12/12 [00:11<00:00,  1.01it/s]


In [38]:
result

{'context_precision': 1.0000, 'context_recall': 0.7786, 'faithfulness': 0.9785, 'answer_relevancy': 0.9219}

In [39]:
df = result.to_pandas()
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,VKS có những tính năng gì nổi bật?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,"Được rồi, mình cần回答 câu hỏi ""VKS có những tín...",\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.0,0.785714,1.000000,0.928116
1,VKS là gì?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,"Okay, so I'm trying to figure out what VKS is....",\n VKS là viết tắt của VNGCloud Kubernetes ...,1.0,0.800000,0.935484,0.902287
2,So sánh public và private cluster trong VKS,[3. Mô hình hoạt động của VKS\nVNGCloud Kubern...,"Được rồi, mình cần so sánh public cluster và p...",\n Public Cluster và Private Cluster là hai...,1.0,0.750000,1.000000,0.935439


In [40]:
def highlight_cells(value):
    """Color cells based on score value."""
    if value < 0.5:
        return "background-color: red; color: white;"  # Low Score (Bad)
    elif 0.5 <= value < 0.8:
        return "background-color: orange; color: black;"  # Medium Score (Warning)
    else:
        return "background-color: lightgreen; color: black;"  # High Score (Good)


def truncate_text(value):
    """Truncate text values longer than 200 characters."""

    if isinstance(value, str) and len(value) > 200:
        return value[:200]

    if isinstance(value, list):
        return "\n\n\n\n".join(v[:100] for v in value)

    return value


# Apply styling
styled_df = df.map(truncate_text)  # Truncate text fields
styled_df = styled_df.style.map(highlight_cells,
                                subset=["context_precision", "context_recall", "faithfulness", "answer_relevancy"])

# Display in Jupyter Notebook
styled_df


,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,VKS có những tính năng gì nổi bật?,"1. Giới thiệu về VNGCloud Kubernetes Service - VKS 1.1. VNGCloud Kubernetes Service (VKS) là gì? VNG 3. Mô hình hoạt động của VKS VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public cho các tài nguyên Service có thuộc tính serviceType là LoadBalancer . Mặc định, một cụm Kubern VNGCloud Portal. vks.vngcloud.vn/scheme : Bắt buộc phải có trong spec của tài nguyên Service : Khô","Được rồi, mình cần回答 câu hỏi ""VKS có những tính năng gì nổi bật?"". Đầu tiên mình sẽ xem qua các tài liệu được cung cấp để tìm thông tin về VKS. Trong Document đầu tiên, có mục 1.2 nêu các điểm nổi bậ",Dịch vụ VKS (VNGCloud Kubernetes Service) có những tính năng nổi bật sau: Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS giúp bạn giải phóng khỏi gánh nặng quản l,1.000000,0.785714,1.000000,0.928116
1,VKS là gì?,"1. Giới thiệu về VNGCloud Kubernetes Service - VKS 1.1. VNGCloud Kubernetes Service (VKS) là gì? VNG 3. Mô hình hoạt động của VKS VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public cho các tài nguyên Service có thuộc tính serviceType là LoadBalancer . Mặc định, một cụm Kubern VNGCloud Portal. vks.vngcloud.vn/scheme : Bắt buộc phải có trong spec của tài nguyên Service : Khô","Okay, so I'm trying to figure out what VKS is. Let's see, I have some context documents provided. Starting with the first document, it mentions that VNGCloud Kubernetes Service, or VKS, is a managed K",VKS là viết tắt của VNGCloud Kubernetes Service. Nó là một dịch vụ quản lý Kubernetes được cung cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một,1.000000,0.800000,0.935484,0.902287
2,So sánh public và private cluster trong VKS,"3. Mô hình hoạt động của VKS VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public dụ: vks.vngcloud.vn/security-groups: secg-303d9c91-f4b2-400d-8e81-12039167da0c . Ngoài ra, bạn cũng VNGCloud Portal. vks.vngcloud.vn/scheme : Bắt buộc phải có trong spec của tài nguyên Service : Khô Security Group cho NLB của bạn bằng cách sử dụng dấu phẩy , để ngăn cách giữa các Security Group.","Được rồi, mình cần so sánh public cluster và private cluster trong VKS. Đầu tiên, mình sẽ đọc các thông tin được cung cấp trong documents để tìm hiểu về hai loại cluster này. Trong document đầu tiên,","Public Cluster và Private Cluster là hai loại cụm Kubernetes chính được hỗ trợ bởi VKS. Mỗi loại có ưu nhược điểm riêng, phù hợp với những nhu cầu bảo mật và sử dụng khác nhau. Dưới đây là b",1.000000,0.750000,1.000000,0.935439
